<img src="aiayn.png">

> When teaching, I emphasize implementation as a way to understand recent developments in ML. This post is an attempt to keep myself honest along this goal. The recent ["Attention is All You Need"]
(https://arxiv.org/abs/1706.03762) paper from NIPS 2017 has been instantly impactful paper as a new method for machine translation and potentiall NLP generally. The paper is very clearly written, but the conventional wisdom has been that it is quite difficult to implement correctly.
>
> In this post I follow the paper through from start to finish and try to implement each component in code.
(I have done some minor reordering and skipping from the original paper). This document itself is a working notebook, and should be a completely usable and efficient implementation. To follow along you will first need to install [PyTorch](http://pytorch.org/) and [torchtext](https://github.com/pytorch/text). The complete code is available on [github](https://github.com/harvardnlp/annotated-transformer).
>- Alexander "Sasha" Rush ([@harvardnlp](https://twitter.com/harvardnlp))


In [ ]:
!pip install transformers datasets

* Table of Contents                               
{:toc}      

In [ ]:
# Standard PyTorch imports
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math, copy
from torch.autograd import Variable

# For plots
%matplotlib inline
import matplotlib.pyplot as plt


# Background

The goal of reducing sequential computation also forms the foundation of the Extended Neural GPU
[16], ByteNet [18] and ConvS2S [9], all of which use convolutional neural networks as basic building
block, computing hidden representations in parallel for all input and output positions. In these models,
the number of operations required to relate signals from two arbitrary input or output positions grows
in the distance between positions, linearly for ConvS2S and logarithmically for ByteNet. This makes
it more difficult to learn dependencies between distant positions [12]. In the Transformer this is
reduced to a constant number of operations, albeit at the cost of reduced effective resolution due
to averaging attention-weighted positions, an effect we counteract with Multi-Head Attention as
described in section 3.2.

Self-attention, sometimes called intra-attention is an attention mechanism relating different positions
of a single sequence in order to compute a representation of the sequence. Self-attention has been
used successfully in a variety of tasks including reading comprehension, abstractive summarization,
textual entailment and learning task-independent sentence representations [4, 27, 28, 22].
End-to-end memory networks are based on a recurrent attention mechanism instead of sequencealigned
recurrence and have been shown to perform well on simple-language question answering and
language modeling tasks [34].

To the best of our knowledge, however, the Transformer is the first transduction model relying
entirely on self-attention to compute representations of its input and output without using sequencealigned
RNNs or convolution. In the following sections, we will describe the Transformer, motivate
self-attention and discuss its advantages over models such as [17, 18] and [9].

# Model Architecture

Most competitive neural sequence transduction models have an encoder-decoder structure [(cite)](cho2014learning,bahdanau2014neural,sutskever14). Here, the encoder maps an input sequence of symbol representations $(x_1, ..., x_n)$ to a sequence of continuous representations $\mathbf{z} = (z_1, ..., z_n)$. Given $\mathbf{z}$, the decoder then generates an output sequence $(y_1,...,y_m)$ of symbols one element at a time. At each step the model is auto-regressive [(cite)](graves2013generating), consuming the previously generated symbols as additional input when generating the next.

In [ ]:
class EncoderDecoder(nn.Module):
    """
    A standard Encoder-Decoder architecture. Base model for this and many
    other models.
    """
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator

    def forward(self, src, tgt, src_mask, tgt_mask):
        "Take in and process masked src and target sequences."
        memory = self.encoder(self.src_embed(src), src_mask)
        output = self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)
        return output

The Transformer follows this overall architecture using stacked self-attention and point-wise, fully connected layers for both the encoder and decoder, shown in the left and right halves of Figure 1, respectively.

<img src="ModalNet-21.png" width=400px>

## Encoder and Decoder Stacks   

### Encoder:

The encoder is composed of a stack of $N=6$ identical layers.

In [ ]:
def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [ ]:
class Encoder(nn.Module):
    "Core encoder is a stack of N layers"
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)

    def forward(self, x, mask):
        "Pass the input (and mask) through each layer in turn."
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

We employ a residual connection [(cite)](he2016deep) around each of the two sub-layers, followed by layer normalization [(cite)](layernorm2016).  

In [ ]:
class LayerNorm(nn.Module):
    "Construct a layernorm module (See citation for details)."
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

That is, the output of each sub-layer is $\mathrm{LayerNorm}(x + \mathrm{Sublayer}(x))$, where $\mathrm{Sublayer}(x)$ is the function implemented by the sub-layer itself.  We apply dropout [(cite)](srivastava2014dropout) to the output of each sub-layer, before it is added to the sub-layer input and normalized.  

To facilitate these residual connections, all sub-layers in the model, as well as the embedding layers, produce outputs of dimension $d_{\text{model}}=512$.  

In [ ]:
class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    Note for code simplicity we apply the norm first as opposed to last.
    """
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer function that maintains the same size."
        return x + self.dropout(sublayer(self.norm(x)))

Each layer has two sub-layers. The first is a multi-head self-attention mechanism, and the second is a simple, position-wise fully connected feed-forward network.
在编码层的内部有两个部分：第一个部分为多头自注意力层，第二个部分为一般的前馈神经网络层（以全连接层进行实现）

In [ ]:
class EncoderLayer(nn.Module):
    "Encoder is made up of two sublayers, self-attn and feed forward (defined below)"
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask):
        "Follow Figure 1 (left) for connections."
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)

### Decoder:

The decoder is also composed of a stack of $N=6$ identical layers.  


In [ ]:
class Decoder(nn.Module):
    "Generic N layer decoder with masking."
    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)

    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)

In addition to the two sub-layers in each encoder layer, the decoder inserts a third sub-layer, which performs multi-head attention over the output of the encoder stack.  Similar to the encoder, we employ residual connections around each of the sub-layers, followed by layer normalization.  
以下代码的说明(待更新和补充)
- m表示从编码器的结果的输出，将会作为K，V的输入
- 每一层有两个attention block
    - 第一个为解码层内部的attention，一般会采用因果的自注意力机制，通过mask相结合进行实现
    - 第二个为和编码层的attention的输出进行交叉自注意力机制的实现，这里的mask为src的mask，因为src有部分采用了padding，只需计算有信息部分的自注意力的相关矩阵的部分


In [ ]:
class DecoderLayer(nn.Module):
    "Decoder is made up of three sublayers, self-attn, src-attn, and feed forward (defined below)"
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 3)

    def forward(self, x, memory, src_mask, tgt_mask):
        "Follow Figure 1 (right) for connections."
        m = memory
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))
        return self.sublayer[2](x, self.feed_forward)

We also modify the self-attention sub-layer in the decoder stack to prevent positions from attending to subsequent positions.  This masking, combined with fact that the output embeddings are offset by one position, ensures that the predictions for position $i$ can depend only on the known outputs at positions less than $i$.

亦即因果(causual self attention block)自注意力机制的实现
创建一个注意力掩码（Attention Mask）。这个特殊的掩码通常被称为**“后续掩码” (Subsequent Mask)** 或 “前瞻掩码” (Look-ahead Mask)
在自然语言生成任务中，比如机器翻译，模型在生成第 i 个词时，只能依赖于它已经生成的前 i-1 个词，而不能“偷看”未来的词（即第 i 个、第 i+1 个词等）。
在计算自注意力（Self-Attention）时，它会**遮挡（mask out）**掉所有未来的位置，确保每个位置的输出只依赖于当前和之前的位置。
np.ones(attn_shape): 创建一个形状为 (1, size, size)、所有元素都为 1 的 NumPy 数组。
np.triu (triangle upper) 的作用是保留一个矩阵的上三角部分，将其余部分置为 0。参数 k 控制着对角线的位置：
- k=0 (默认): 保留主对角线及以上部分。
- k=1: 保留主对角线上方的部分，主对角线本身也被置为 0。
- k=-1: 保留主对角线下方部分。

torch.from_numpy(subsequent_mask) == 0有类似取反的作用
- 原来是 0 的位置，比较后变为 True。
- 原来是 1 的位置，比较后变为 False。

scores 是注意力分数张量，形状通常是 (batch_size, num_heads, seq_len, seq_len)  
scores = scores.masked_fill(mask == False, -1e9)  
将掩码中为 False 的位置（即我们想遮挡的未来位置）对应的注意力分数替换成一个非常大的负数（如 -1e9）。当这个带有极大负数的张量经过 Softmax 函数时，这些位置的概率会趋近于 0，从而有效地“忽略”了它们。

In [ ]:
def subsequent_mask(size):
    "Mask out subsequent positions."
    attn_shape = (1, size, size)
    subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')
    return torch.from_numpy(subsequent_mask) == 0

In [ ]:
# The attention mask shows the position each tgt word (row) is allowed to look at (column).
# Words are blocked for attending to future words during training.
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

plt.figure(figsize=(5,5))
plt.imshow(subsequent_mask(20)[0])

### Attention:                                                                                                                                                                                                                                                                               
An attention function can be described as mapping a query and a set of key-value pairs to an output, where the query, keys, values, and output are all vectors.  The output is computed as a weighted sum of the values, where the weight assigned to each value is computed by a compatibility function of the query with the corresponding key.                                                                                                                                                                                                                                                                                           

We call our particular attention "Scaled Dot-Product Attention".   The input consists of queries and keys of dimension $d_k$, and values of dimension $d_v$.  We compute the dot products of the query with all keys, divide each by $\sqrt{d_k}$, and apply a softmax function to obtain the weights on the values.                                                                                                         
<img width="220px" src="ModalNet-19.png">

In practice, we compute the attention function on a set of queries simultaneously, packed together into a matrix $Q$.   The keys and values are also packed together into matrices $K$ and $V$.  We compute the matrix of outputs as:                      
                                                                 
$$                                                                         
   \mathrm{Attention}(Q, K, V) = \mathrm{softmax}(\frac{QK^T}{\sqrt{d_k}})V               
$$       

自注意力机制的代码实现，这里用到了mask，mask为0的地方将不参与计算（经过softmax计算后其概率比重为0）

torch.matmul 的内部实现比这更高效，它使用的是一种叫做**批处理矩阵乘法（Batch Matrix Multiplication）**的机制

当 torch.matmul(tensor1, tensor2) 的输入张量维度大于 2 时，它遵循以下规则：
最后两维是矩阵：torch.matmul 将每个张量的最后两个维度视为要进行乘法运算的矩阵。
前面所有维度是“批次”维度：除最后两维之外的所有维度都被视为“批次”维度。torch.matmul 会在这些批次维度上进行广播（broadcasting）。
独立计算：它会把运算看作是一批独立的矩阵乘法。对于批次维度中的每一个组合，它都会执行一次标准的 2D 矩阵乘法。
关键在于：这个“批处理”操作不是通过 Python 循环实现的，而是在底层（如 C++ 或 CUDA）通过高度优化的代码执行的，能够充分利用硬件的并行计算能力。

以[batch, multi-head-count, tokens_capacity, d_k] 形状为例来分析。为了方便，我们用符号表示：[B, H, T, Dk]。  
B: batch_size (批次大小)  
H: multi-head-count (头的数量)  
T: tokens_capacity (序列长度)  
Dk: d_k (每个头的维度)  
计算 scores  scores = torch.matmul(query, key.transpose(-2, -1))    
query 的形状: (B, H, T, Dk)  
key 的形状: (B, H, T, Dk)  
key.transpose(-2, -1) 的形状:  
transpose(-2, -1) 会交换最后两个维度。  
所以，形状变为 (B, H, Dk, T)。  
现在，我们来看 torch.matmul 的输入：  
tensor1 (query): (B, H, T, Dk)  
tensor2 (key.transpose(...)): (B, H, Dk, T)    
因此，scores 的最终形状是 (B, H, T, T)，这正是我们期望的注意力分数矩阵的形状（对于每个头和每个批次项）

In [ ]:
def attention(query, key, value, mask=None, dropout=0.0):
    "Compute 'Scaled Dot Product Attention'"
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) \
             / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim = -1)
    # (Dropout described below)
    p_attn = F.dropout(p_attn, p=dropout)
    return torch.matmul(p_attn, value), p_attn

The two most commonly used attention functions are additive attention [(cite)](bahdanau2014neural), and dot-product (multiplicative) attention.  Dot-product attention is identical to our algorithm, except for the scaling factor of $\frac{1}{\sqrt{d_k}}$. Additive attention computes the compatibility function using a feed-forward network with a single hidden layer.  While the two are similar in theoretical complexity, dot-product attention is much faster and more space-efficient in practice, since it can be implemented using highly optimized matrix multiplication code.                                                                                             

                                                                        
While for small values of $d_k$ the two mechanisms perform similarly, additive attention outperforms dot product attention without scaling for larger values of $d_k$ [(cite)](DBLP:journals/corr/BritzGLL17). We suspect that for large values of $d_k$, the dot products grow large in magnitude, pushing the softmax function into regions where it has extremely small gradients  (To illustrate why the dot products get large, assume that the components of $q$ and $k$ are independent random variables with mean $0$ and variance $1$.  Then their dot product, $q \cdot k = \sum_{i=1}^{d_k} q_ik_i$, has mean $0$ and variance $d_k$.). To counteract this effect, we scale the dot products by $\frac{1}{\sqrt{d_k}}$.          

### Multi-Head Attention                                                                                                                                                                                                                                                                                                                                                                                                                               
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
Instead of performing a single attention function with $d_{\text{model}}$-dimensional keys, values and queries, we found it beneficial to linearly project the queries, keys and values $h$ times with different, learned linear projections to $d_k$, $d_k$ and $d_v$ dimensions, respectively.                                                                                                                                                                                                   
On each of these projected versions of queries, keys and values we then perform the attention function in parallel, yielding $d_v$-dimensional output values. These are concatenated and once again projected, resulting in the final values:

<img width="270px" src="ModalNet-20.png">
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
Multi-head attention allows the model to jointly attend to information from different representation subspaces at different positions. With a single attention head, averaging inhibits this.                                                                                                                                                                                                                                                                                             
    
    
   
$$    
\mathrm{MultiHead}(Q, K, V) = \mathrm{Concat}(\mathrm{head_1}, ..., \mathrm{head_h})W^O    \\                                           
    \text{where}~\mathrm{head_i} = \mathrm{Attention}(QW^Q_i, KW^K_i, VW^V_i)                                
$$                                                                                                                                                                                                                                                                                                                                                                         
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
Where the projections are parameter matrices $W^Q_i \in \mathbb{R}^{d_{\text{model}} \times d_k}$, $W^K_i \in \mathbb{R}^{d_{\text{model}} \times d_k}$, $W^V_i \in \mathbb{R}^{d_{\text{model}} \times d_v}$ and $W^O \in \mathbb{R}^{hd_v \times d_{\text{model}}}$.                                                                                                                                                                                                                                                        
   

   
In this work we employ $h=8$ parallel attention layers, or heads. For each of these we use $d_k=d_v=d_{\text{model}}/h=64$. Due to the reduced dimension of each head, the total computational cost is similar to that of single-head attention with full dimensionality.   


多头自注意力机制的实现，h为head的数目，d_model为模型的输出的维度(batch_size, token_capacity, d_model)
```
def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

query, key, value = [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
                    for l, x in zip(self.linears, (query, key, value))]
```

zip 函数在打包多个可迭代对象（如列表、元组）时，会以其中最短的那个为准。 一旦最短的可迭代对象耗尽，zip 就会停止。
self.linears: 这是一个 nn.ModuleList，通过 clones(..., 4) 创建，其长度为 4。
[Linear_Layer_0, Linear_Layer_1, Linear_Layer_2, Linear_Layer_3]
(query, key, value): 这是一个元组，包含了三个张量，其长度为 3。
因此，zip(self.linears, (query, key, value)) 只会产生三对元素：
- (self.linears[0], query)
- (self.linears[1], key)
- (self.linears[2], value)
self.linears 中的第四个线性层 (self.linears[3]) 在这个列表推导中不会被使用。而在后续融合的使用
```
self.linears[-1](x)
```

关于attention的高阶张量的计算说明在上面已有提及。
下面这句代码将多个head的结果合并到到一起，形成[B,T,d_model]的张量输出   
最后再通过线性层进行一次融合后形成最终的张量输出[B,T,d_model]  
```
x = x.transpose(1, 2).contiguous().view(nbatches, -1, self.h * self.d_k)\
```

In [ ]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        "Take in model size and number of heads."
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.p = dropout
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None

    def forward(self, query, key, value, mask=None):
        "Implements Figure 2"
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)

        # 1) Do all the linear projections in batch from d_model => h x d_k
        query, key, value = [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
                             for l, x in zip(self.linears, (query, key, value))]

        # 2) Apply attention on all the projected vectors in batch.
        x, self.attn = attention(query, key, value, mask=mask, dropout=self.p)

        # 3) "Concat" using a view and apply a final linear.
        x = x.transpose(1, 2).contiguous().view(nbatches, -1, self.h * self.d_k)
        return self.linears[-1](x)

### Applications of Attention in our Model                                                                                                                                                      
The Transformer uses multi-head attention in three different ways:                                                        
1) In "encoder-decoder attention" layers, the queries come from the previous decoder layer, and the memory keys and values come from the output of the encoder.   This allows every position in the decoder to attend over all positions in the input sequence.  This mimics the typical encoder-decoder attention mechanisms in sequence-to-sequence models such as [(cite)](wu2016google, bahdanau2014neural,JonasFaceNet2017).    


2) The encoder contains self-attention layers.  In a self-attention layer all of the keys, values and queries come from the same place, in this case, the output of the previous layer in the encoder.   Each position in the encoder can attend to all positions in the previous layer of the encoder.                                                   


3) Similarly, self-attention layers in the decoder allow each position in the decoder to attend to all positions in the decoder up to and including that position.  We need to prevent leftward information flow in the decoder to preserve the auto-regressive property.  We implement this inside of scaled dot-product attention by masking out (setting to $-\infty$) all values in the input of the softmax which correspond to illegal connections.                                                                                                                                                                                                                                                      

## Position-wise Feed-Forward Networks                                                                                                                                                                                                                                                                                                                                                             
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
In addition to attention sub-layers, each of the layers in our encoder and decoder contains a fully connected feed-forward network, which is applied to each position separately and identically.  This consists of two linear transformations with a ReLU activation in between.

$$\mathrm{FFN}(x)=\max(0, xW_1 + b_1) W_2 + b_2$$                                                                                                                                                                                                                                                         
                                                                                                                                                                                                                                                        
While the linear transformations are the same across different positions, they use different parameters from layer to layer. Another way of describing this is as two convolutions with kernel size 1.  The dimensionality of input and output is $d_{\text{model}}=512$, and the inner-layer has dimensionality $d_{ff}=2048$.

全连接前馈层，两层全连接网络

In [ ]:
class PositionwiseFeedForward(nn.Module):
    "Implements FFN equation."
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        # Torch linears have a `b` by default.
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))

## Embeddings and Softmax                                                                                                                                                                                                                                                                                           
Similarly to other sequence transduction models, we use learned embeddings to convert the input tokens and output tokens to vectors of dimension $d_{\text{model}}$.  We also use the usual learned linear transformation and softmax function to convert the decoder output to predicted next-token probabilities.  In our model, we share the same weight matrix between the two embedding layers and the pre-softmax linear transformation, similar to [(cite)](press2016using). In the embedding layers, we multiply those weights by $\sqrt{d_{\text{model}}}$.              

嵌入层，从输入到transformer的模型维度d_model

In [ ]:
class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)

## Positional Encoding                                                                                                                             
Since our model contains no recurrence and no convolution, in order for the model to make use of the order of the sequence, we must inject some information about the relative or absolute position of the tokens in the sequence.  To this end, we add "positional encodings" to the input embeddings at the bottoms of the encoder and decoder stacks.  The positional encodings have the same dimension $d_{\text{model}}$ as the embeddings, so that the two can be summed.   There are many choices of positional encodings, learned and fixed [(cite)](JonasFaceNet2017).

In this work, we use sine and cosine functions of different frequencies:                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
$$                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
    PE_{(pos,2i)} = sin(pos / 10000^{2i/d_{\text{model}}}) \\                                                                                                                                                                                                                                                                                                                                                                                                                                      
    PE_{(pos,2i+1)} = cos(pos / 10000^{2i/d_{\text{model}}})                                                                                                                                                                                                                                                                                                                                                                                                                                       
$$                                                                                                                                                                                                                                                        
where $pos$ is the position and $i$ is the dimension.  That is, each dimension of the positional encoding corresponds to a sinusoid.  The wavelengths form a geometric progression from $2\pi$ to $10000 \cdot 2\pi$.  We chose this function because we hypothesized it would allow the model to easily learn to attend by relative positions, since for any fixed offset $k$, $PE_{pos+k}$ can be represented as a linear function of $PE_{pos}$.

In addition, we apply dropout to the sums of the embeddings and the positional encodings in both the encoder and decoder stacks.  For the base model, we use a rate of $P_{drop}=0.1$.

位置编码，         

$$
e^{2i*(-log(10000)/d)}= e^{-log(10000)*(2i/d)}=\frac{1}{e^{log(10000)^{(2i/d)}}}=\frac{1}{(10000)^{(2i/d)}}
$$


In [ ]:
class PositionalEncoding(nn.Module):
    "Implement the PE function."
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) *
                             -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)], requires_grad=False)
        return self.dropout(x)

如下代码说明：d_model=20,T=100

In [ ]:
# The positional encoding will add in a sine wave based on position.
# The frequency and offset of the wave is different for each dimension.
plt.figure(figsize=(15, 5))
pe = PositionalEncoding(20, 0)
y = pe.forward(Variable(torch.zeros(1, 100, 20)))
plt.plot(np.arange(100), y[0, :, 4:8].data.numpy())
plt.legend(["dim %d"%p for p in [4,5,6,7]])
None

We also experimented with using learned positional embeddings [(cite)](JonasFaceNet2017) instead, and found that the two versions produced nearly identical results.  We chose the sinusoidal version because it may allow the model to extrapolate to sequence lengths longer than the ones encountered during training.    

## Generation

In [ ]:
class Generator(nn.Module):
    "Standard generation step. (Not described in the paper.)"
    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        return F.log_softmax(self.proj(x), dim=-1)

## Full Model

In [ ]:
def make_model(src_vocab, tgt_vocab, N=6, d_model=512, d_ff=2048, h=8, dropout=0.1):
    "Construct a model object based on hyperparameters."
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model, dropout)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    model = EncoderDecoder(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), c(ff), dropout), N),
        nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
        nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
        Generator(d_model, tgt_vocab))

    # This was important from their code. Initialize parameters with Glorot or fan_avg.
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform(p)
    return model

# Training

This section describes the training regime for our models.


## Training Data and Batching

We trained on the standard WMT 2014 English-German dataset consisting of about 4.5 million sentence pairs.  Sentences were encoded using byte-pair encoding \citep{DBLP:journals/corr/BritzGLL17}, which has a shared source-target vocabulary of about 37000 tokens. For English-French, we used the significantly larger WMT 2014 English-French dataset consisting of 36M sentences and split tokens into a 32000 word-piece vocabulary [(cite)](wu2016google).


Sentence pairs were batched together by approximate sequence length.  Each training batch contained a set of sentence pairs containing approximately 25000 source tokens and 25000 target tokens.     

## Hardware and Schedule                                                                                                                                                                                                   
We trained our models on one machine with 8 NVIDIA P100 GPUs.  For our base models using the hyperparameters described throughout the paper, each training step took about 0.4 seconds.  We trained the base models for a total of 100,000 steps or 12 hours.  For our big models, step time was 1.0 seconds.  The big models were trained for 300,000 steps (3.5 days).

## Optimizer

We used the Adam optimizer [(cite)](kingma2014adam) with $\beta_1=0.9$, $\beta_2=0.98$ and $\epsilon=10^{-9}$.  We varied the learning rate over the course of training, according to the formula:                                                                                            
$$                                                                                                                                                                                                                                                                                         
lrate = d_{\text{model}}^{-0.5} \cdot                                                                                                                                                                                                                                                                                                
  \min({step\_num}^{-0.5},                                                                                                                                                                                                                                                                                                  
    {step\_num} \cdot {warmup\_steps}^{-1.5})                                                                                                                                                                                                                                                                               
$$                                                                                                                                                                                             
This corresponds to increasing the learning rate linearly for the first $warmup\_steps$ training steps, and decreasing it thereafter proportionally to the inverse square root of the step number.  We used $warmup\_steps=4000$.           

在前面的warmup_steps的训练步骤里，线性的增加学习率，然后随着步骤的平方根的倒数成正比进行衰减

In [ ]:
# Note: This part is incredibly important.
# Need to train with this setup of the model is very unstable.
class NoamOpt:
    "Optim wrapper that implements rate."
    def __init__(self, model_size, factor, warmup, optimizer):
        self.optimizer = optimizer
        self._step = 0
        self.warmup = warmup
        self.factor = factor
        self.model_size = model_size
        self._rate = 0

    def step(self):
        "Update parameters and rate"
        self._step += 1
        rate = self.rate()
        for p in self.optimizer.param_groups:
            p['lr'] = rate
        self._rate = rate
        self.optimizer.step()

    def rate(self, step = None):
        "Implement `lrate` above"
        if step is None:
            step = self._step
        return self.factor * \
            (self.model_size ** (-0.5) *
            min(step ** (-0.5), step * self.warmup**(-1.5)))

def get_std_opt(model):
    return NoamOpt(model.src_embed[0].d_model, 2, 4000,
            torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))

In [ ]:
# Three settings of the lrate hyperparameters.
opts = [NoamOpt(512, 1, 4000, None),
        NoamOpt(512, 1, 8000, None),
        NoamOpt(256, 1, 4000, None)]
plt.plot(np.arange(1, 20000), [[opt.rate(i) for opt in opts] for i in range(1, 20000)])
plt.legend(["512:4000", "512:8000", "256:4000"])
None

## Regularization                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
                                                                                                                                                                                                                                                                                                                      
### Label Smoothing

During training, we employed label smoothing of value $\epsilon_{ls}=0.1$ [(cite)](DBLP:journals/corr/SzegedyVISW15).  This hurts perplexity, as the model learns to be more unsure, but improves accuracy and BLEU score.  

In [ ]:
class LabelSmoothing(nn.Module):
    "Implement label smoothing."
    def __init__(self, size, pad_token_id, smoothing=0.0):
        super(LabelSmoothing, self).__init__()
        #self.criterion = nn.KLDivLoss(size_average=False)
        self.criterion = nn.KLDivLoss(reduction='sum')
        self.pad_token_id = pad_token_id
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.size = size #vocab_size
        self.true_dist = None

    def forward(self, x, target): #shape of x=[batch_size, vocab_size]，一个字符一个字符计算loss
        assert x.size(1) == self.size
        true_dist = x.data.clone()
        true_dist.fill_(self.smoothing / (self.size - 2))#pad和目标标签不参与计算，亦即将真值位置之外的概率分布(smoothing)平摊到其他的token位置
        true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)#将token真值的位置设为confidence，真值定义为soft label
        true_dist[:, self.pad_token_id] = 0 #pad的token id为0，故true_dist(i)loginput(i)=0×loginput(i)=0，为0的位置在将不参与loss的计算，，也就是pad的token在vocab_size的divloss中不参与loss的计算
        mask = torch.nonzero(target.data == self.pad_token_id)
        if mask.dim() > 0:
            true_dist.index_fill_(0, mask.squeeze(), 0.0)#从0维度进行填充，如果其对应的mask不为0，则这一行所有的分布为0，也就是token为pad的所有vocab_size都不参与loss的计算
        self.true_dist = true_dist
        return self.criterion(x, Variable(true_dist, requires_grad=False))

LabelSmoothing使用示例 LabelSmoothing(5, 0, 0.5)，5为vocab_size，词汇表大小，0为pad的token id，0.5为smoothing因子
predict为三个预测的输出的概率分布，crit即根据LabelSmoothing进行loss计算

```
torch.LongTensor([2, 1, 0]))-->

tensor([[0.0000, 0.1667, 0.5000, 0.1667, 0.1667],
        [0.0000, 0.5000, 0.1667, 0.1667, 0.1667],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000]])
```

In [ ]:
#Example
crit = LabelSmoothing(5, 0, 0.5)
predict = torch.FloatTensor([[0, 0.2, 0.7, 0.1, 0],
                             [0, 0.2, 0.7, 0.1, 0],
                             [0, 0.2, 0.7, 0.1, 0]])
v = crit(Variable(predict.log()),
         Variable(torch.LongTensor([2, 1, 0])))

# Show the target distributions expected by the system.
plt.imshow(crit.true_dist)
None

随着预测的准确度的增加，loss的下降可视化图示

In [ ]:
# Label smoothing starts to penalize the model
# if it gets very confident about a given choice
# crit = LabelSmoothing(5, 0, 0.2)
crit = nn.KLDivLoss(reduction='batchmean')
def loss(x):
    d = x + 3 * 1
    predict = torch.FloatTensor([[1e-9, x / d, 1 / d, 1 / d, 1 / d],
                                 ])
    target = torch.FloatTensor([[0, 1, 0, 0, 0]])  # 使用one-hot向量作为目标分布
    return crit(predict.log(), target).item()
    #print(predict)
    # return crit(Variable(predict.log()),
    #              Variable(torch.LongTensor([1]))).item()
print(loss(10.0))
plt.plot(np.arange(1, 100), [loss(x) for x in range(1, 100)])

### Memory Optimization

输出out的张量形状为out->[batch, max_seq_len， d_model],Variable(out[:, i].data为第i个token
用时间换空间：通过一个循环，一次只处理序列中的一个时间步（一个 token），手动计算和累积梯度，从而避免一次性创建和存储那些巨大的中间张量。


In [ ]:
def loss_backprop(generator, criterion, out, targets, valid_token_counts):
    """
    Memory optmization. Compute each timestep separately and sum grads.
    """
    assert out.size(1) == targets.size(1)
    total_loss = 0.0
    total = 0.0
    out_grad = []
    #print(out.shape)torch.Size([2, 127, 512])
    for i in range(out.size(1)): #out->[batch, max_seq_len， dmodel] for each token，这个循环遍历序列的每一个时间步（token），从 0到 T-1
        # 将这个切片出的数据重新包装成一个新的叶子节点（leaf node）变量 out_column，并明确告诉 PyTorch 我们需要计算它相对于后续操作的梯度。这样做是为了构建一个临时的、局部的计算图。
        # 1. Prepare a slice of the output tensor for this time step
        # This slice is detached from the main graph to create a local computation graph.
        out_column = Variable(out[:, i].data, requires_grad=True)
        
        # 2. Generate predictions for this time step
        gen_out = generator(out_column) # Shape: [batch_size, vocab_size]
        
        # 3. Calculate the loss for this time step
        # Note: criterion should be defined with reduction='none' to get per-element loss.
        # e.g., criterion = nn.CrossEntropyLoss(reduction='none', ignore_index=padding_idx)
        loss = criterion(gen_out, targets[:, i]) # Shape: [batch_size]
        
        # 4. Create a mask to filter out padding tokens for this time step
        # A token is valid if its index 'i' is less than the total valid length.
        mask = (i < valid_token_counts).float() # Shape: [batch_size]
        
        # 5. Apply the mask to the loss
        masked_loss = loss * mask # Shape: [batch_size]
        
        # 6. Sum the loss for valid tokens and perform backpropagation
        # We only backpropagate the loss from non-padded tokens.
        # loss.sum() will correctly handle cases where some samples in the batch are masked out.
        loss_sum = masked_loss.sum()
        
        # Accumulate total loss for reporting (normalized by the total number of valid tokens)
        total_loss += loss_sum.item()

        # Perform backpropagation on the local graph to get gradient w.r.t. out_column
        # If loss_sum is 0 (e.g., all samples are padding at this step), no gradient is computed.
        if loss_sum > 0:
            loss_sum.backward()

        # 7. Store the gradient
        # If a sample was masked, its gradient will be zero, which is correct.
        # We need to handle the case where out_column.grad is None if loss_sum was 0.
        grad_data = out_column.grad.data.clone() if out_column.grad is not None else torch.zeros_like(out_column.data)
        out_grad.append(grad_data)            
    out_grad = torch.stack(out_grad, dim=1)
    #这是连接主计算图并完成整个反向传播过程的魔法。
    # 当在一个张量（非标量）上调用 .backward() 时，你需要提供一个 gradient 参数。
    # 将我们手动计算并拼接好的梯度 out_grad (即 dL/d(out)) 作为起始梯度，从 out 节点开始，沿着主计算图继续向后传播，
    # 从而计算出损失相对于模型中所有更早参数（如 Encoder、Decoder 的权重）的梯度。
    out.backward(gradient=out_grad)
    return total_loss

In [ ]:
# 'input_ids': source['input_ids'].squeeze(),
# 'src_atten_mask': source['attention_mask'].squeeze(),
# 'tgt_atten_mask': target['attention_mask'].squeeze(),
# 'labels': target['input_ids'].squeeze()  # 目标是英语的tokenized形式
def train_epoch(train_iter, model, criterion, opt, transpose=False):
    model.train()
    for i, batch in enumerate(train_iter):
        opt.optimizer.zero_grad()
        # src, trg, src_mask, trg_mask = \
        #     batch.src, batch.trg, batch.src_mask, batch.trg_mask
        #print(batch.keys())
        #[2,1,128]
        #print(batch['input_ids'].shape,batch['labels'].shape, batch['src_atten_mask'].shape, batch['tgt_atten_mask'].shape)
        src, trg, src_mask, trg_mask = \
            batch['input_ids'], batch['labels'], batch['src_atten_mask'], batch['tgt_atten_mask']
        #out = model.forward(src, trg[:, :-1], src_mask, trg_mask[:, :-1])
        out = model.forward(src, trg, src_mask, trg_mask)
        loss = loss_backprop(model.generator, criterion, out[:,:-1,:], trg[:, 1:], batch['valid_token_count'])

        opt.step()

        if i % 10 == 1:
            print(i, loss, opt._rate)

In [ ]:
def valid_epoch(valid_iter, model, criterion, transpose=False):
    model.test()
    total = 0
    for batch in valid_iter:
        src, trg, src_mask, trg_mask = \
            batch.src, batch.trg, batch.src_mask, batch.trg_mask
        out = model.forward(src, trg[:, :-1], src_mask, trg_mask[:, :-1, :-1])
        loss = loss_backprop(model.generator, criterion, out, trg[:, 1:], batch.ntokens)


# A Real World Example

In [ ]:

# from datasets import load_dataset

# # 加载 IWSLT2017 数据集的德语到英语的翻译数据集
# dataset = load_dataset('iwslt2017', 'iwslt2017-de-en',trust_remote_code=True)

# # 查看数据集的结构
# print(dataset)

# # 数据集已经分为训练集、验证集和测试集
# train_dataset = dataset['train']
# validation_dataset = dataset['validation']
# test_dataset = dataset['test']


# # 示例：查看训练集中第一条数据
# example = train_dataset[0]
# print(f"德语: {example['translation']['de']}")
# print(f"英语: {example['translation']['en']}")
from datasets import load_dataset

# 加载 IWSLT2017 数据集的德语到英语的翻译数据集
dataset = load_dataset('iwslt2017', 'iwslt2017-zh-en',trust_remote_code=True)

# 查看数据集的结构
print(dataset)

# 数据集已经分为训练集、验证集和测试集
train_dataset = dataset['train']
validation_dataset = dataset['validation']
test_dataset = dataset['test']


# 示例：查看训练集中第一条数据
example = train_dataset[0]
print(f"中文: {example['translation']['zh']}")
print(f"英语: {example['translation']['en']}")


非常谢谢，克里斯。的确非常荣幸 能有第二次站在这个台上的机会，我真是非常感激。

torch.Size([1, 128]) tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]])

In [ ]:
from transformers import AutoTokenizer

# 使用预训练的BERT tokenizer
#tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')
#BERT基于wordpiece
#GPT基于BPE
ch_tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")
en_tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


# 获取 [CLS] 和 [PAD] 的 token ID
cls_token_id = en_tokenizer.convert_tokens_to_ids(en_tokenizer.cls_token)
pad_token_id = en_tokenizer.convert_tokens_to_ids(en_tokenizer.pad_token)

print(f"[CLS] Token ID: {cls_token_id}")
print(f"[PAD] Token ID: {pad_token_id}")

# 特殊标记
bos_token = "[CLS]"  # 在BERT中，通常使用[CLS]作为开始标记
sep_token = "[SEP]"  # 在BERT中，通常使用[SEP]作为结束标记

# 定义最大序列长度
max_seq_length = 128
source = ch_tokenizer(
    example['translation']['zh'],
    padding='max_length',
    truncation=True,
    max_length=max_seq_length,
    return_tensors='pt'
)

target = en_tokenizer(
    f"{bos_token} {example['translation']['en']} {sep_token}",
    padding='max_length',
    truncation=True,
    max_length=max_seq_length,
    return_tensors='pt'
)

print(source['input_ids'].squeeze().shape, source['attention_mask'].squeeze().shape)
print(source['attention_mask'].shape, source['attention_mask'])
mask = source['attention_mask'].squeeze()
#[128, 1],[1,128]
att_mask = mask.unsqueeze(-1)@mask.unsqueeze(0)
print(att_mask.shape, att_mask)

In [ ]:
def subsequent_mask_v2(size):
    "Mask out subsequent positions."
    attn_shape = (size, size)
    subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')
    return torch.from_numpy(subsequent_mask) == 0

src_mask基于有效的token进行计算
tgt_mask基于有效的token和causual masked的交操作进行计算

In [ ]:
def make_std_mask(src_mask, tgt_mask, max_seq_len):
    src_mask = src_mask.unsqueeze(-1)@src_mask.unsqueeze(0)
    tgt_mask = tgt_mask.unsqueeze(-1)@tgt_mask.unsqueeze(0)
    tgt_mask = tgt_mask & Variable(subsequent_mask_v2(max_seq_len).type_as(tgt_mask.data))
    return src_mask, tgt_mask

src_mask, tgt_mask = make_std_mask(source['attention_mask'].squeeze(), target['attention_mask'].squeeze(), max_seq_length)
print(src_mask.shape, tgt_mask.shape, target['attention_mask'].sum(dim=1))

In [ ]:
# 定义编码函数
def encode_example(example):
    source = ch_tokenizer(
        example['translation']['zh'],
        padding='max_length',
        truncation=True,
        max_length=max_seq_length,
        return_tensors='pt'
    )
    target = en_tokenizer(
        f"{bos_token} {example['translation']['en']} {sep_token}",
        padding='max_length',
        truncation=True,
        max_length=max_seq_length,
        return_tensors='pt'
    )
    src_mask, tgt_mask = make_std_mask(source['attention_mask'].squeeze(), target['attention_mask'].squeeze(), max_seq_length)
    # 计算有效 token 的数目
    valid_token_count = target['attention_mask'].sum(dim=1)
    return {
        'input_ids': source['input_ids'].squeeze(),
        'src_atten_mask': src_mask,
        'tgt_atten_mask': tgt_mask,
        'valid_token_count':valid_token_count,
        'labels': target['input_ids'].squeeze()  # 目标是英语的tokenized形式
    }

# 对数据集进行tokenization
encoded_train_dataset = train_dataset.map(encode_example)
encoded_validation_dataset = validation_dataset.map(encode_example)
encoded_test_dataset = test_dataset.map(encode_example)

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class TranslationDataset(Dataset):
    def __init__(self, encoded_dataset):
        self.dataset = encoded_dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.dataset[idx]['input_ids'], dtype=torch.long),
            'src_atten_mask': torch.tensor(self.dataset[idx]['src_atten_mask'], dtype=torch.long),
            'tgt_atten_mask': torch.tensor(self.dataset[idx]['tgt_atten_mask'], dtype=torch.long),
            'labels': torch.tensor(self.dataset[idx]['labels'], dtype=torch.long),
            'valid_token_count': torch.tensor(self.dataset[idx]['valid_token_count'], dtype=torch.long),
        }

# 创建 PyTorch Dataset
train_data = TranslationDataset(encoded_train_dataset)
validation_data = TranslationDataset(encoded_validation_dataset)
test_data = TranslationDataset(encoded_test_dataset)

# 创建 DataLoader
batch_size = 16
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
validation_dataloader = DataLoader(validation_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)


In [ ]:
#引文有补齐时常用0，其代表的padding的含义
print(ch_tokenizer.pad_token_id, en_tokenizer.pad_token_id, ch_tokenizer.vocab_size, en_tokenizer.vocab_size)

In [ ]:
# Create the model an load it onto our GPU.
#pad_idx = TGT.vocab.stoi["<blank>"]
# model = make_model(len(SRC.vocab), len(TGT.vocab), N=6)
# model_opt = get_std_opt(model)
# model.cuda()
model = make_model(ch_tokenizer.vocab_size, en_tokenizer.vocab_size, N=6)
model_opt = get_std_opt(model)
#model.cuda()

In [ ]:

criterion = LabelSmoothing(size=en_tokenizer.vocab_size, pad_token_id=en_tokenizer.pad_token_id, smoothing=0.1)
#criterion.cuda()
for epoch in range(15):
    train_epoch(train_dataloader, model, criterion, model_opt)
    valid_epoch(validation_dataloader, model, criterion)


OTHER

In [ ]:
BOS_WORD = '<s>'
EOS_WORD = '</s>'
BLANK_WORD = "<blank>"
SRC = data.Field()
TGT = data.Field(init_token = BOS_WORD, eos_token = EOS_WORD, pad_token=BLANK_WORD) # only target needs BOS/EOS

MAX_LEN = 100
train = datasets.TranslationDataset(path="/n/home00/srush/Data/baseline-1M_train.tok.shuf",
                                    exts=('.en', '.fr'),
                                    fields=(SRC, TGT),
                                    filter_pred=lambda x: len(vars(x)['src']) <= MAX_LEN and
                                         len(vars(x)['trg']) <= MAX_LEN)
SRC.build_vocab(train.src, max_size=50000)
TGT.build_vocab(train.trg, max_size=50000)

In [ ]:
pad_idx = TGT.vocab.stoi["<blank>"]
print(pad_idx)
model = make_model(len(SRC.vocab), len(TGT.vocab), pad_idx, N=6)
model_opt = get_opt(model)
model.cuda()

In [ ]:
criterion = LabelSmoothing(size=len(TGT.vocab), padding_idx=pad_idx, label_smoothing=0.1)
criterion.cuda()
for epoch in range(15):
    train_epoch(train_iter, model, criterion, model_opt)
    valid_epoch()

In [ ]:
print(pad_idx)
print(len(SRC.vocab))

In [ ]:
torch.save(model, "/n/rush_lab/trans_ipython.pt")

In [ ]:
#weight = torch.ones(len(TGT.vocab))
#weight[pad_idx] = 0
#criterion = nn.NLLLoss(size_average=False, weight=weight.cuda())
criterion = LabelSmoothing(size=len(TGT.vocab), padding_idx=pad_idx, label_smoothing=0.1)
criterion.cuda()
for epoch in range(15):
    train_epoch(train_iter, model, criterion, model_opt)